# Tutorial 08: modify a linear system on restriction

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **penalty imposition of Dirichlet BCs**: this requires first the discretization of the associated homogeneous Neumann problem
$$
\text{find } u \in V \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V\\
$$
where
$$
V = H^1(\Omega),
$$
which we rewrite in matrix form as
$$ \text{find } x \in \mathbb{R}^n \text{ s.t. } A x = b,$$
where $A \in \mathbb{R}^{n \times n}$ and $b \in \mathbb{R}^n$ are the left-hand side and right-hand side of the discrete Neumann problem.

In order to impose Dirichlet boundary conditions, we solve the modified system
$$ \text{find } \tilde{x} \in \mathbb{R}^n \text{ s.t. } \tilde{A} \tilde{x} = \tilde{b},$$
where
$$\tilde{A} = A + P, \qquad \tilde{b} = b + q.$$

The penalty matrix $P$ is defined as
$$
P_{ij} =
\begin{cases}
\mu, & \text{if $i = j$, and $i$ is a boundary DOF},\\
0, & \text{otherwise},
\end{cases}$$
and the penalty vector $q$ as
$$
q_i = 
\begin{cases}
\mu g(\mathbf{x}_i), & \text{if $i$ is a boundary DOF},\\
0, & \text{otherwise},
\end{cases}$$
where $\mu \in \mathbb{R}$ is (large) scalar number, and $\mathbf{x}_i$ denotes the coordinate of the entity associated at DOF $i$.

The preferred way to impose non-homogeneous Dirichlet boundary conditions should still
either be through DirichletBC objects (see tutorial 01) or lagrange multipliers
(see tutorial 03). This example is meant to show how to use the list of degrees of freedom
associated to a specific restriction to perform local modifications to assembled
matrices.

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import grad, inner, Measure, TestFunction, TrialFunction
from dolfinx.cpp.la import create_petsc_vector
from dolfinx.fem import DirichletBC, Function, FunctionSpace, LinearProblem, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import assemble_matrix, assemble_scalar, assemble_vector, create_matrix
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
facets_Gamma = boundaries.indices[boundaries.values == 1]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Helper functions

In [ ]:
def generate_penalty_system(V, restriction, penalty, g):
    """
    Generate matrix and vector to be added to the system to handle the penalty terms
    """
    # Assemble penalty matrix.
    p = u * v * ds  # this form is not really used per se, just to allocate the correct sparsity pattern
    P = create_matrix(p)  # this creates a zero matrix
    for dof in restriction:
        P.setValuesLocal([dof], [dof], [penalty])  # set P_{ij} = penalty * \delta_{ij}, if i on boundary
    P.assemble()
    # Next, assemble the penalty vector.
    q = create_petsc_vector(V.dofmap.index_map, V.dofmap.index_map_bs)
    with q.localForm() as q_local, g.vector.localForm() as g_local:
        q_local[restriction] = penalty * g_local[restriction]
    # Return matrix and vector
    return P, q

### Penalty imposition of Dirichlet BCs

In [ ]:
# Define a function space
V = FunctionSpace(mesh, ("Lagrange", 2))

In [ ]:
# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

In [ ]:
# Define problem forms
g = Function(V)
g.interpolate(lambda x: np.sin(3 * x[0] + 1) * np.sin(3 * x[1] + 1))
a = inner(grad(u), grad(v)) * dx
f = v * dx

In [ ]:
# Assemble the linear system
A = assemble_matrix(a)
A.assemble()
F = assemble_vector(f)
F.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

In [ ]:
# Add penalty terms
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
(P, Q) = generate_penalty_system(V, dofs_V_Gamma, 1.e10, g)

In [ ]:
u = Function(V)
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A + P)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.solve(F + Q, u.vector)
u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_scalar_field_plot(mesh, u, "u")

### Strong imposition of Dirichlet BCs

In [ ]:
# Define Dirichlet BC object on Gamma
bc_ex = DirichletBC(g, dofs_V_Gamma)

In [ ]:
# Solve
u_ex = Function(V)
problem_ex = LinearProblem(
    a, f, bcs=[bc_ex], u=u_ex,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
problem_ex.solve()
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
pyvista_scalar_field_plot(mesh, u_ex, "u")

### Comparison and error computation

In [ ]:
u_ex_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(grad(u_ex), grad(u_ex)) * dx), op=MPI.SUM))
err_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(grad(u_ex - u), grad(u_ex - u)) * dx), op=MPI.SUM))
print("Relative error is equal to", err_norm / u_ex_norm)
assert np.isclose(err_norm / u_ex_norm, 0., atol=1.e-10)